# TP1 - NER

### En utilisant vos connaissances en programmation, codez un système NER qui reconnaît les noms de personnes (PERSON) et les lieux (LOCATION) à partir de l'ensemble de données fourni (europarl).

#### 1 - Division du texte en phrases et en mots (c'est-à-dire en jetons)

In [5]:
import glob
import json
import re
import nltk

from nltk.corpus import stopwords


# 1 - On enlève la ponctuation
def enlever_ponctuation(chaine):
    return re.sub(r"[^\w\s]", "", chaine)

# 2 - On enlève les mots vides
def enlever_mots_vides(chaine):
    return [mot for mot in chaine if mot not in stopwords.words("french")]



_
